# https://www.mongodb.com/docs/atlas/atlas-vector-search/create-embeddings/

In [1]:
import sys
from Data.KEY import NOTES # type: ignore
import pymongo

ModuleNotFoundError: No module named 'KEY'

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
sentences = ['search_document: TSNE is a dimensionality reduction algorithm created by Laurens van Der Maaten']
embeddings = model.encode(sentences)
print(embeddings)


In [ ]:
from sentence_transformers import SentenceTransformer

# Load the embedding model (https://huggingface.co/nomic-ai/nomic-embed-text-v1")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

# Define a function to generate embeddings
def get_embedding(data):
   """Generates vector embeddings for the given data."""

   embedding = model.encode(data)
   return embedding.tolist()

# Generate an embedding
get_embedding("foo")

In [ ]:
# Filter to exclude null or empty content fields
filter = { "content": {"$nin": [ None, "" ]} }

# Get a subset of documents in the NOTES
documents = NOTES.find(filter).limit(50)

# Update each document with a new embedding field
updated_doc_count = 0
for doc in documents:
    embedding = get_embedding(doc["summary"])
    NOTES.update_one( { "_id": doc["_id"] }, { "$set": { "embedding": embedding } } )
    updated_doc_count += 1
print(f"Updated {updated_doc_count} documents.")

In [ ]:
# Generate embedding for the search query
query_embedding = get_embedding("beach house")
# Sample vector search pipeline
pipeline = [
   {
      "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "exact": True,
            "limit": 5
      }
   }, 
   {
      "$project": {
         "_id": 1,
         "title": 1,
         "desciption": 1,
         "content": 1,
         "score": {
            "$meta": "vectorSearchScore"
         }
      }
   }
]
# Execute the search
results = NOTES.aggregate(pipeline)
# Print results
for i in results:
   print(i)